In [178]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

LC = pd.read_csv("C:\\Users\\skyamz\\Desktop\\ppdai_3_213\\LC.csv") 
LP = pd.read_csv("C:\\Users\\skyamz\\Desktop\\ppdai_3_213\\LP.csv")

In [179]:
#把日期都转换程日期格式，方便计算
LP['到期日期'] = pd.to_datetime(LP['到期日期'])
LP['还款日期'] = LP['还款日期'].replace('\\N','2017-02-22')#将还款日期不明确的标的，还款日期统一设置为记录日期，即2017-02-22                
LP['还款日期'] = pd.to_datetime(LP['还款日期'])
LC['orn_rank'] = LC['初始评级']#为了防止中文乱码的问题，把’初始评级‘改成’orn_rank‘

In [180]:
#计算逾期，并将其转换成int格式，为了以后计算方便
LP['day_diff'] =  LP['还款日期'] - LP['到期日期']
LP['day_diff'] = LP['day_diff'].astype(np.str) #先转换成string格式，然后利用map函数来逐一变成数字
Tran_Date = lambda day_diff: int(day_diff.split('days',1)[0])
LP['day_diff']= LP['day_diff'].map(Tran_Date)

In [181]:
LP02 = LP[(LP['还款状态']==2)|(LP['还款状态']==0)]#选择已经逾期的标的和未还款的标的

In [182]:
LP02 = LP02[(LP02['day_diff']<=90)&(LP02['day_diff']>=1)] #催收回款的定义为逾期 90 天之内的逾期还款                                        

In [183]:
LP02 = pd.merge(LC,LP02,on='ListingId')#内联LC与LP

In [184]:
#根据期数和逾期，对逾期和未还的标的进行分类
lable = ['期数','day_diff']
level = pd.DataFrame(LP02.pivot_table(index=lable,columns=['还款状态'],values=['应还本金'],aggfunc=np.sum))

In [185]:
level

应还本金            
还款状态        0           2
期数 day_diff                      
1  1         29676.68  5682916.70
   2         43575.52  1716821.04
   3         27975.91   963314.59
   4          7942.23  1801022.51
   5          8910.97   741656.28
   6          8581.46   448885.56
   7          5094.73   493035.53
   8          9345.21   235856.47
   9          5186.78   163851.21
   10         4111.40   152670.49
   11         2502.65   103789.73
   12         2743.39    88975.42
   13         3648.13    71408.78
   14         1807.27    47834.43
   15         2496.14    41187.31
   16         3812.61    89340.94
   17         3380.28   107352.69
   18         4029.45    49936.41
   19         2215.34    38175.70
   20         2052.53    33652.95
   21         1702.32    25351.02
   22         1121.25    32816.41
   23         3205.93    33046.09
   24         2769.23    51998.28
   25         2778.75    15215.53
   26         2310.39    17590.77
   27         3904.07    15280.26
   28         2871.27    16757.04
   29         1857.72     7372.57
   30         1520.91    11622.87
...               ...         ...
20 9              NaN     1037.73
   12         1037.73         NaN
   14         1037.73         NaN
   20         1853.70     1405.76
   21         1554.17         NaN
   22         1874.35         NaN
   24         1234.70         NaN
   28         1234.70         NaN
   29         1230.90         NaN
   32        19360.39         NaN
   33         2891.43         NaN
   36         1037.73         NaN
   37         1558.96         NaN
   45         1234.70         NaN
   55         1037.73         NaN
   57         2272.43         NaN
   60         1037.73         NaN
21 1         11135.52     7926.99
   2          2933.08         NaN
   3              NaN     1646.32
   4              NaN      705.57
   5          1051.57         NaN
   6          1579.74         NaN
   14         1251.16         NaN
   24         2103.14         NaN
   26         2302.73         NaN
   29         1051.57         NaN
22 6              NaN     1065.59
   9              NaN     1065.59
23 8              NaN      726.90

[1451 rows x 2 columns]

In [186]:
level.columns= ['0','2']
level = level.fillna(0)
#计算催收回款率（back_rate） = 催收回本金(2)/所有逾期本金(total)
level['total'] = level['0'] + level['2']
level['back_rate'] = level['2']/level['total']
level['back_rate'] = level['back_rate'].astype(np.str)#将back_rate变为str，方便输出
level = level.reset_index()#将level标中的标签变换为列名

In [187]:
level

,期数,day_diff,0,2,total,back_rate
0,1,1,29676.68,5682916.70,5712593.38,0.994805042469
1,1,2,43575.52,1716821.04,1760396.56,0.975246759173
2,1,3,27975.91,963314.59,991290.50,0.971778293043
3,1,4,7942.23,1801022.51,1808964.74,0.995609516413
4,1,5,8910.97,741656.28,750567.25,0.988127686093
5,1,6,8581.46,448885.56,457467.02,0.98124135812
6,1,7,5094.73,493035.53,498130.26,0.989772293697
7,1,8,9345.21,235856.47,245201.68,0.961887659171
8,1,9,5186.78,163851.21,169037.99,0.969315891653
9,1,10,4111.40,152670.49,156781.89,0.973776307965


In [190]:
rank  = list(level['期数'].unique())#取初始评级
day_diff,back_rate = [],[]
#根据不同的初始评级，将逾期和催收回款率存入list中，方便画图
for i in rank:
        temp = level[level['期数']==i]
        day_diff.append(list(temp['day_diff']))
        back_rate.append(list(temp['back_rate']))

In [192]:
len(day_diff)

23

In [193]:
back_rate

[['0.994805042469',
  '0.975246759173',
  '0.971778293043',
  '0.995609516413',
  '0.988127686093',
  '0.98124135812',
  '0.989772293697',
  '0.961887659171',
  '0.969315891653',
  '0.973776307965',
  '0.976455038451',
  '0.970089123485',
  '0.951395148028',
  '0.963593712544',
  '0.942858450969',
  '0.959071769138',
  '0.969473590386',
  '0.92533334964',
  '0.945152687329',
  '0.942514986495',
  '0.937075422111',
  '0.966961481729',
  '0.911565479662',
  '0.949436627665',
  '0.845575927461',
  '0.883906767244',
  '0.796496932653',
  '0.853717920697',
  '0.798736551073',
  '0.884286712042',
  '0.881712356211',
  '0.941028721251',
  '0.800067622878',
  '0.846385218356',
  '0.710920803293',
  '0.819531317203',
  '0.740441638961',
  '0.725516223699',
  '0.535696201356',
  '0.726724881839',
  '0.592947000833',
  '0.677846601936',
  '0.793227609515',
  '0.825551477342',
  '0.732871882474',
  '0.720790745117',
  '0.455727080373',
  '0.546754720117',
  '0.523117647693',
  '0.832742711577',
  

In [194]:
#绘制初始评级对催收回款率影响的折线图
fig = plt.figure(figsize = (20,10))#设置大小
plt.title('orn_rank influnence')#标题
plt.xlabel('day_diff')#X轴
plt.ylabel('back_rate')#Y轴
#分别对应['A','B','C','D','E','F'] 的颜色
colors = ['red','blue','green','orange','black','purple']
#对于每个初始评级，绘制它们对应的催收回款率
for i in range(0,23):
        plt.plot(day_diff[i],back_rate[i],c = colors[i],label = rank[i])
        plt.legend(loc='lower left')#标签位置

IndexError: list index out of range

In [176]:
plt.show()